## Building a Nuclei Classifier Model from Extracted Feature Data

In this comprehensive tutorial, we will guide you through the process of constructing a nuclei classifier using the powerful *Random Forest* algorithm. Our approach involves extracting features from image data using DSA and assigning classes to the data instances using K-Means method. A foundational grasp of Python and familiarity with utilizing iPython notebooks are prerequisites for comprehending the content presented in this tutorial.

**Included Resources**
- [Input Image](https://data.kitware.com/api/v1/file/hashsum/sha512/1ff135eb0ff8864a876a19ae3dec579f27f1718726a68643f6a40a244fdfa08e81f63f1413c198b38384cb34e8705bc60a6c69ef2b706cb0419f6ec091b2b621/download)
- [Extracted Features file (Optional)](https://data.kitware.com/api/v1/file/hashsum/sha512/e8c829b60d316ff84d2ffafb5accd605eb8dcd02dec709105ec9127aa2d7969e2feca74f66394b26f0e90375cd0d1cda3d1831023449f66cf50a637906444578/download)

*This tutorial is created by Subin Erattakulangara (Kitware)*

### Step 1 (Extract nuclei features)

Open the Nuclei Feature extraction panel in DSA. and upload the image data into *Input Image* area shown below. Provide the location for both feature file and annotation files to be saved. Then press submit to start the process.The cli will generate the feature file requried for the classifier. Annotation file is not required for creating the classfier.

![DSA panel.png](https://data.kitware.com/api/v1/file/hashsum/sha512/10f88a5400e7fa46605e3f75530ae8703a429fbbf1185444a14fa40beec251434d19760de90bdaae25b5ece3557b502b59e40fab377b3df5978088b14c3a14e2/download)

### Step 2 (Generate training labels)

Once the CLI generates the feature file, download it. Create a new folder and put the downloaded feature file in there. Then, run the provided Python code within the same folder. This code enhances the feature file with classes. This simple flow ensures you manage, organize, and improve your feature file effortlessly.<br><br>
You can also use the `.csv` file provided above to create the classifier.

##### Rad the csv file

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pickle


In [ ]:
# Read CSV file
import pandas as pd
df = pd.read_csv('<Input feature filename>.csv')
df.head()


##### Standardize the data and perform K-means clustering

In [ ]:
# Standardize the data
X = df.values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Perform KMeans clustering
num_clusters = 5  # Number of clusters you want to create
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(X_scaled)


##### Add generated cluster labels to feature file

In [ ]:
# Add cluster labels to the original data
df['Cluster'] = cluster_labels

# Print the count of data points in each cluster
print(df['Cluster'].value_counts())


In this process we have modified the feature file so that the class labels are also added into it. These labels are required to train the Random forest classifier.

### Step 3 (Train random forest classifier)

##### Convert dataframe to target and labels

In [ ]:
# Assuming the last column contains the target labels
X = df.iloc[:, :-1]  # Features
y = df.iloc[:, -1]   # Target labels
print(X.shape, y.shape)

# Convert categorical labels to numerical using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


##### Split the data into training and testing sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


##### Train a random forest classifier

In [ ]:
# Create a RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = classifier.predict(X_test)

# Convert numerical predictions back to categorical labels
y_pred_labels = label_encoder.inverse_transform(y_pred)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


#### Save the trained model into a pickle file

In [ ]:
model_filename = '<output file name>.pkl'
with open(model_filename, 'wb') as model_file:
    pickle.dump(classifier, model_file)

print(f"Model saved as {model_filename}")


This trained model can be used for nuclei classfication - [link to the nuclei classification tutorial]()